In [1]:
from cnn.model import build_net
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

2022-12-22 12:22:31.092660: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


3 Physical GPUs, 1 Logical GPU


2022-12-22 12:22:32.835715: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 12:22:33.451086: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22122 MB memory:  -> device: 1, name: NVIDIA A30, pci bus id: 0000:af:00.0, compute capability: 8.0


In [2]:
unet_net_list = ["vgg_n_3","vgg_d_3","vgg_d_3","vgg_d_3","vgg_d_3","vgg_u_3","vgg_u_3","vgg_u_3","vgg_u_3","vgg_n_3"]
resunet_net_list = ["res_n_3","res_d_3","res_d_3","res_d_3","res_d_3","res_u_3","res_u_3","res_u_3","res_u_3","res_n_3"]

experiment_1_spleen_net_list = ["vgg_n_3","vgg_d_3","vgg_u_3","vgg_d_3","vgg_d_3","vgg_d_3","vgg_d_3","vgg_d_3","vgg_n_3","vgg_u_3"]
experiment_2_spleen_net_list = ["den_n_3","res_d_5","vgg_d_5","res_d_5","inc_d_3","res_u_3","res_u_3","res_u_5","inc_u_3","ide_n"]
experiment_3_spleen_net_list = ["res_n_3","den_d_3","res_u_7","inc_d_7","vgg_d_3","vgg_d_3","res_d_5","inc_d_7","res_n_7","den_u_3"]
experiment_4_spleen_net_list = ["inc_n_3","den_d_3","den_d_3","res_u_5","res_n_3","res_n_5","res_d_5","den_n_5","den_n_5","den_d_3"]

experiment_1_prostate_net_list = ["vgg_u_3","vgg_u_3","vgg_d_3","vgg_d_3","vgg_d_3","vgg_d_3","vgg_d_3","vgg_u_3","vgg_u_3","vgg_n_3"]
experiment_2_prostate_net_list = ["ide_n","vgg_d_3","vgg_d_5","inc_d_3","den_d_3","den_u_5","vgg_u_5","res_u_5","inc_u_7","vgg_n_7"]
experiment_3_prostate_net_list = ["vgg_u_5","inc_u_5","vgg_d_7","res_d_3","inc_d_3","ide_d","den_d_7","vgg_u_7","res_u_5","den_n_5"]
experiment_4_prostate_net_list = ['vgg_d_7',"vgg_d_7","inc_d_5","den_d_5","res_d_5","vgg_u_7","vgg_d_5","vgg_n_5","inc_n_5","den_d_7"]

layer_dict = {
    "den_3":   {                           "block": "DenseBlock",     "kernel": 3},
    "den_5":   {                           "block": "DenseBlock",     "kernel": 5},
    "den_7":   {                           "block": "DenseBlock",     "kernel": 7},
    "den_d_3": {"cell": "DownscalingCell", "block": "DenseBlock",     "kernel": 3},
    "den_d_5": {"cell": "DownscalingCell", "block": "DenseBlock",     "kernel": 5},
    "den_d_7": {"cell": "DownscalingCell", "block": "DenseBlock",     "kernel": 7},
    "den_n_3": {"cell": "NonscalingCell",  "block": "DenseBlock",     "kernel": 3},
    "den_n_5": {"cell": "NonscalingCell",  "block": "DenseBlock",     "kernel": 5},
    "den_n_7": {"cell": "NonscalingCell",  "block": "DenseBlock",     "kernel": 7},
    "den_u_3": {"cell": "UpscalingCell",   "block": "DenseBlock",     "kernel": 3},
    "den_u_5": {"cell": "UpscalingCell",   "block": "DenseBlock",     "kernel": 5},
    "den_u_7": {"cell": "UpscalingCell",   "block": "DenseBlock",     "kernel": 7},
    "inc_3":   {                           "block": "InceptionBlock", "kernel": 3},
    "inc_5":   {                           "block": "InceptionBlock", "kernel": 5},
    "inc_7":   {                           "block": "InceptionBlock", "kernel": 7},
    "inc_d_3": {"cell": "DownscalingCell", "block": "InceptionBlock", "kernel": 3},
    "inc_d_5": {"cell": "DownscalingCell", "block": "InceptionBlock", "kernel": 5},
    "inc_d_7": {"cell": "DownscalingCell", "block": "InceptionBlock", "kernel": 7},
    "inc_n_3": {"cell": "NonscalingCell",  "block": "InceptionBlock", "kernel": 3},
    "inc_n_5": {"cell": "NonscalingCell",  "block": "InceptionBlock", "kernel": 5},
    "inc_n_7": {"cell": "NonscalingCell",  "block": "InceptionBlock", "kernel": 7},
    "inc_u_3": {"cell": "UpscalingCell",   "block": "InceptionBlock", "kernel": 3},
    "inc_u_5": {"cell": "UpscalingCell",   "block": "InceptionBlock", "kernel": 5},
    "inc_u_7": {"cell": "UpscalingCell",   "block": "InceptionBlock", "kernel": 7},
    "ide":     {                           "block": "IdentityBlock"              },
    "ide_d":   {"cell": "DownscalingCell", "block": "IdentityBlock"              },
    "ide_n":   {"cell": "NonscalingCell",  "block": "IdentityBlock"              },
    "ide_u":   {"cell": "UpscalingCell",   "block": "IdentityBlock"              },
    "res_3":   {                           "block": "ResNetBlock",    "kernel": 3},
    "res_5":   {                           "block": "ResNetBlock",    "kernel": 5},
    "res_7":   {                           "block": "ResNetBlock",    "kernel": 7},
    "res_d_3": {"cell": "DownscalingCell", "block": "ResNetBlock",    "kernel": 3},
    "res_d_5": {"cell": "DownscalingCell", "block": "ResNetBlock",    "kernel": 5},
    "res_d_7": {"cell": "DownscalingCell", "block": "ResNetBlock",    "kernel": 7},
    "res_n_3": {"cell": "NonscalingCell",  "block": "ResNetBlock",    "kernel": 3},
    "res_n_5": {"cell": "NonscalingCell",  "block": "ResNetBlock",    "kernel": 5},
    "res_n_7": {"cell": "NonscalingCell",  "block": "ResNetBlock",    "kernel": 7},
    "res_u_3": {"cell": "UpscalingCell",   "block": "ResNetBlock",    "kernel": 3},
    "res_u_5": {"cell": "UpscalingCell",   "block": "ResNetBlock",    "kernel": 5},
    "res_u_7": {"cell": "UpscalingCell",   "block": "ResNetBlock",    "kernel": 7},
    "vgg_3":   {                           "block": "VGGBlock",       "kernel": 3},
    "vgg_5":   {                           "block": "VGGBlock",       "kernel": 5},
    "vgg_7":   {                           "block": "VGGBlock",       "kernel": 7},
    "vgg_d_3": {"cell": "DownscalingCell", "block": "VGGBlock",       "kernel": 3},
    "vgg_d_5": {"cell": "DownscalingCell", "block": "VGGBlock",       "kernel": 5},
    "vgg_d_7": {"cell": "DownscalingCell", "block": "VGGBlock",       "kernel": 7},
    "vgg_n_3": {"cell": "NonscalingCell",  "block": "VGGBlock",       "kernel": 3},
    "vgg_n_5": {"cell": "NonscalingCell",  "block": "VGGBlock",       "kernel": 5},
    "vgg_n_7": {"cell": "NonscalingCell",  "block": "VGGBlock",       "kernel": 7},
    "vgg_u_3": {"cell": "UpscalingCell",   "block": "VGGBlock",       "kernel": 3},
    "vgg_u_5": {"cell": "UpscalingCell",   "block": "VGGBlock",       "kernel": 5},
    "vgg_u_7": {"cell": "UpscalingCell",   "block": "VGGBlock",       "kernel": 7},
}

In [3]:
patch_size = (128, 128)
batch_size = 32
num_classes = 3
stem_filters = 16
max_depth = 4

In [4]:
model = build_net((*patch_size, 2), stem_filters, max_depth, num_classes, layer_dict, experiment_4_prostate_net_list)
model.summary()

Model: "net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 128, 128, 2  0           []                               
                                )]                                                                
                                                                                                  
 StemConvolution_Convolution_Co  (None, 128, 128, 16  288        ['input[0][0]']                  
 nvolution_3x3 (Conv2D)         )                                                                 
                                                                                                  
 StemConvolution_Normalization_  (None, 128, 128, 16  64         ['StemConvolution_Convolution_Con
 Normalization (BatchNormalizat  )                               volution_3x3[0][0]']           

In [5]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

plot_model(
        model,
        to_file='model.png',
        show_shapes=True,
        show_layer_names=True,
        rankdir='TB'
)

Image('model.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


FileNotFoundError: No such file or directory: 'model.png'

FileNotFoundError: No such file or directory: 'model.png'

<IPython.core.display.Image object>